In [1]:
%load_ext autoreload
%autoreload 2

# IMPORTS

In [2]:
import os
from dotenv import load_dotenv

# data
import pandas as pd

# gensim
import gensim
from gensim.corpora.dictionary import Dictionary

# spacy
from spacy.lang.de.stop_words import STOP_WORDS
STOP_WORDS = list(STOP_WORDS)

# scikit
from sklearn.feature_extraction.text import TfidfVectorizer # ,CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
# from sklearn.datasets import fetch_20newsgroups

# topicwizard
import topicwizard
from topicwizard.pipeline import make_topic_pipeline
# from topicwizard.compatibility import gensim_pipeline

# pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models

# src
from src.general.io import to_pickle, from_pickle
from src.general.io import disable_warnings
from src.models.custom_lda import CustomLDA

In [3]:
disable_warnings()

# PATHS & NAMES

In [4]:
load_dotenv()

RAW_DATA_PATHS = {}
for short_name, filename in zip(["feb", "mar", "apr", "may", "feedback"],
                                ["event_stg_user_input_web_cw6.txt",
                                "event_stg_user_input_web_cw10.txt",
                                "event_stg_user_input_web_cw15.txt",
                                "event_stg_user_input_web_cw19.txt",
                                "current_user_feedback_text.xlsx",
                                ]):
    RAW_DATA_PATHS[short_name] = f"..{os.getenv('RAW_DATA_DIR')}/{filename}"


EXT_DATA_PATHS = {}
for short_name, filename in zip(["rasa"],
                                ["rasa_train_data.pkl"]):
    EXT_DATA_PATHS[short_name] = f"..{os.getenv('EXTERNAL_DATA_DIR')}/{filename}"


INTERIM_DATA_PATHS = {}
for short_name, filename in zip(["rasa_docs", "rasa_emb"],
                                ["rasa_docs.pkl",
                                "rasa_embedings.pkl",
                                ]):
    INTERIM_DATA_PATHS[short_name] = f"..{os.getenv('INTERIM_DATA_DIR')}/{filename}"

# add more interims
for short_name in RAW_DATA_PATHS.keys():
    INTERIM_DATA_PATHS[f"{short_name}_doc"] = f"..{os.getenv('INTERIM_DATA_DIR')}/{short_name}_docs.pkl"
    INTERIM_DATA_PATHS[f"{short_name}_emb"] = f"..{os.getenv('INTERIM_DATA_DIR')}/{short_name}_embeding.pkl"
    INTERIM_DATA_PATHS[f"{short_name}_tok"] = f"..{os.getenv('INTERIM_DATA_DIR')}/{short_name}_tokens.pkl"


EDA_REPORT_PATHS = {}
for short_name in RAW_DATA_PATHS.keys():
    EDA_REPORT_PATHS[short_name] = f"..{os.getenv('REPORT_DIR')}/eda_{short_name}.html"

                        
TEXT_COL_USERINPUT = "user_input"
ID_COL_USERINPUT = "session_id"

N_TOPICS = 20
MONTHS = ["feb", "mar", "apr", "may"]

# DATA

In [6]:
token_data = {}
docs_data = {}
for month in MONTHS:
    token_data[month] = from_pickle(INTERIM_DATA_PATHS[f"{month}_tok"])
    docs_data[month] = from_pickle(INTERIM_DATA_PATHS[f"{month}_doc"])

# Topic modelling

## gensim

### LDA

In [7]:
# # Fit LDA model
# gensim_model = gensim.models.ldamodel.LdaModel(
#     corpus = corpus,      # Document-Term Matrix
#     id2word = id2word,    # Map word IDs to words
#     num_topics = N_TOPICS,      # Number of latent topics to extract
#     random_state = SEED,
#     passes = 100,         # N° of passes through the corpus during training
#     )

In [8]:
# # Visualize with pyLDAvis
# pyLDAvis.enable_notebook()
# visualization = pyLDAvis.gensim_models.prepare(gensim_model, corpus, id2word, mds = "mmds", R = 21)
# visualization

### Custom LDA

#### hyper-parameters optimisation

In [11]:
if 1==2:
    custom_lda = CustomLDA(token_data["feb"], seed=int(os.getenv('SEED')))
    custom_lda.grid_search_hyperparameters()
    to_pickle(custom_lda, f"../{os.getenv('MODELS_DIR')}/lda.pkl")
else:
    custom_lda = from_pickle(f"../{os.getenv('MODELS_DIR')}/lda.pkl")

print(f"{custom_lda.best_num_topics=}, {custom_lda.best_alpha=}, {custom_lda.best_beta=}")
custom_lda.df_grid_results.head()

custom_lda.best_num_topics=50, custom_lda.best_alpha=0.61, custom_lda.best_beta=0.61


,Topics,Alpha,Beta,Coherence
0,10,0.01,0.01,0.372782
1,10,0.01,0.31,0.424901
2,10,0.01,0.61,0.471481
3,10,0.01,0.91,0.487000
4,10,0.01,symmetric,0.390388


In [16]:
custom_lda = CustomLDA(token_data["feb"], seed=int(os.getenv('SEED')))

#### Best params

##### Febraury

In [17]:
custom_lda.fit(50, 0.61, 0.61) # when no parameters are provied best (estimated by grid search) would be used
custom_lda.visualize("train")

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
7      0.122086 -0.295445       1        1  8.588720
32     0.141218  0.312399       2        1  7.421220
37    -0.336073  0.041759       3        1  4.342380
38     0.032688  0.056192       4        1  2.074904
42    -0.021641  0.026711       5        1  1.998822
9     -0.002380  0.008810       6        1  1.714657
41     0.001458 -0.003419       7        1  1.678625
29     0.001458 -0.003419       8        1  1.678625
4      0.001459 -0.003418       9        1  1.678625
33     0.001459 -0.003418      10        1  1.678624
23     0.001458 -0.003419      11        1  1.678624
34     0.001459 -0.003418      12        1  1.678624
49     0.001456 -0.003419      13        1  1.678624
30     0.001455 -0.003419      14        1  1.678624
35     0.001457 -0.003419      15        1  1.678624
43     0.001457 -0.003419      16        1  1.678623
1      0.001455 -0.003419      17        1  1.678623
45     0.001457 -0.003419      18        1  1.678623
0      0.001457 -0.003419      19        1  1.678623
26     0.001457 -0.003419      20        1  1.678623
25     0.001458 -0.003419      21        1  1.678622
20     0.001456 -0.003419      22        1  1.678622
8      0.001459 -0.003418      23        1  1.678622
36     0.001455 -0.003419      24        1  1.678621
44     0.001458 -0.003418      25        1  1.678621
18     0.001455 -0.003419      26        1  1.678621
22     0.001455 -0.003419      27        1  1.678621
5      0.001455 -0.003419      28        1  1.678621
31     0.001458 -0.003419      29        1  1.678621
14     0.001456 -0.003419      30        1  1.678621
48     0.001458 -0.003419      31        1  1.678621
24     0.001458 -0.003419      32        1  1.678621
13     0.001459 -0.003418      33        1  1.678621
39     0.001455 -0.003419      34        1  1.678620
46     0.001457 -0.003419      35        1  1.678620
10     0.001456 -0.003419      36        1  1.678620
12     0.001456 -0.003419      37        1  1.678620
11     0.001458 -0.003419      38        1  1.678619
21     0.001456 -0.003419      39        1  1.678617
6      0.001457 -0.003419      40        1  1.678617
15     0.001455 -0.003419      41        1  1.678617
2      0.001456 -0.003419      42        1  1.678616
17     0.001459 -0.003418      43        1  1.678616
27     0.001456 -0.003419      44        1  1.678616
19     0.001458 -0.003419      45        1  1.678616
28     0.001456 -0.003419      46        1  1.678616
3      0.001456 -0.003419      47        1  1.678616
16     0.001458 -0.003419      48        1  1.678615
40     0.001456 -0.003419      49        1  1.678613
47     0.001455 -0.003419      50        1  1.678610, topic_info=          Term        Freq       Total Category  logprob  loglift
7     internet  303.000000  303.000000  Default  21.0000  21.0000
1         kein  279.000000  279.000000  Default  20.0000  20.0000
6          der  391.000000  391.000000  Default  19.0000  19.0000
5        nicht  336.000000  336.000000  Default  18.0000  18.0000
14     storung  212.000000  212.000000  Default  17.0000  17.0000
..         ...         ...         ...      ...      ...      ...
736   anschluß    0.233302   12.486729  Topic50  -7.5101   0.1071
104   leistung    0.231051   12.316729  Topic50  -7.5198   0.1111
213  glasfaser    0.234516   13.533894  Topic50  -7.5049   0.0318
318  telefonie    0.236900   26.838409  Topic50  -7.4948  -0.6427
274        weg    0.231614   15.515412  Topic50  -7.5174  -0.1173

[2369 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
115       1  0.463829             aber
115       2  0.278298             aber
176       4  0.110011         abgehend
162       4  0.192132  anruf_empfangen
84        4  0.795705        anschluss
...     ...       ...              ...
167       3  0.491285       zu_langsam
231       3  0.16870

##### March

In [18]:
custom_lda.add_corpus(token_data["mar"], "mar")
custom_lda.visualize("mar")

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.122190 -0.295449       1        1  11.047843
32     0.142401  0.311768       2        1   5.656591
37    -0.336286  0.040059       3        1   3.861295
9      0.010204  0.007396       4        1   1.851926
38    -0.036256  0.052748       5        1   1.839478
42    -0.004233  0.032679       6        1   1.824640
28     0.002319 -0.003391       7        1   1.679971
40     0.002319 -0.003391       8        1   1.679967
41     0.002319 -0.003391       9        1   1.679967
13     0.002319 -0.003391      10        1   1.679965
31     0.002319 -0.003391      11        1   1.679965
20     0.002320 -0.003391      12        1   1.679964
6      0.002317 -0.003391      13        1   1.679964
30     0.002316 -0.003391      14        1   1.679964
18     0.002318 -0.003391      15        1   1.679964
36     0.002318 -0.003391      16        1   1.679964
14     0.002316 -0.003391      17        1   1.679963
46     0.002318 -0.003391      18        1   1.679963
15     0.002318 -0.003391      19        1   1.679963
33     0.002318 -0.003391      20        1   1.679962
39     0.002319 -0.003391      21        1   1.679962
25     0.002317 -0.003391      22        1   1.679961
43     0.002319 -0.003391      23        1   1.679961
23     0.002316 -0.003391      24        1   1.679961
11     0.002318 -0.003391      25        1   1.679960
1      0.002316 -0.003391      26        1   1.679960
21     0.002316 -0.003391      27        1   1.679960
24     0.002316 -0.003391      28        1   1.679960
10     0.002319 -0.003391      29        1   1.679960
34     0.002317 -0.003391      30        1   1.679959
0      0.002319 -0.003391      31        1   1.679959
29     0.002319 -0.003391      32        1   1.679959
47     0.002320 -0.003391      33        1   1.679959
8      0.002316 -0.003391      34        1   1.679959
19     0.002318 -0.003391      35        1   1.679959
44     0.002317 -0.003391      36        1   1.679958
4      0.002317 -0.003391      37        1   1.679957
35     0.002319 -0.003391      38        1   1.679956
16     0.002317 -0.003391      39        1   1.679956
48     0.002318 -0.003391      40        1   1.679956
3      0.002316 -0.003391      41        1   1.679955
45     0.002317 -0.003391      42        1   1.679955
2      0.002319 -0.003391      43        1   1.679955
27     0.002318 -0.003391      44        1   1.679955
12     0.002319 -0.003391      45        1   1.679955
17     0.002317 -0.003391      46        1   1.679954
5      0.002316 -0.003391      47        1   1.679954
26     0.002319 -0.003391      48        1   1.679953
22     0.002317 -0.003391      49        1   1.679951
49     0.002316 -0.003391      50        1   1.679951, topic_info=          Term       Freq      Total Category  logprob  loglift
6          der  96.000000  96.000000  Default  21.0000  21.0000
7     internet  52.000000  52.000000  Default  20.0000  20.0000
1         kein  48.000000  48.000000  Default  19.0000  19.0000
5        nicht  50.000000  50.000000  Default  18.0000  18.0000
14     storung  52.000000  52.000000  Default  17.0000  17.0000
..         ...        ...        ...      ...      ...      ...
736   anschluß   0.045373   2.437352  Topic50  -7.5101   0.1027
104   leistung   0.044936   2.441016  Topic50  -7.5198   0.0915
213  glasfaser   0.045609   2.730462  Topic50  -7.5049  -0.0057
318  telefonie   0.046074   4.516580  Topic50  -7.4948  -0.4989
274        weg   0.045045   3.106820  Topic50  -7.5174  -0.1473

[2437 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
115       1  0.599343        aber
115       2  0.149836        aber
84        5  0.819238   anschluss
178       2  0.473245        auch
108       3  0.289732     ausfall
...     ...       ...         ...
16        2  0.831814        wlan
282       1  0.542026       woche
82 

##### April

In [20]:
custom_lda.add_corpus(token_data["apr"], "apr")
custom_lda.visualize("apr")

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.122086 -0.295445       1        1  10.561359
32     0.141218  0.312399       2        1   6.004068
37    -0.336073  0.041759       3        1   3.841014
38     0.032688  0.056192       4        1   1.936793
42    -0.021641  0.026711       5        1   1.864857
9     -0.002380  0.008810       6        1   1.829241
29     0.001458 -0.003419       7        1   1.680982
19     0.001458 -0.003419       8        1   1.680980
2      0.001459 -0.003418       9        1   1.680979
49     0.001459 -0.003418      10        1   1.680976
43     0.001458 -0.003419      11        1   1.680976
33     0.001459 -0.003418      12        1   1.680975
0      0.001456 -0.003419      13        1   1.680975
10     0.001455 -0.003419      14        1   1.680975
35     0.001457 -0.003419      15        1   1.680974
18     0.001457 -0.003419      16        1   1.680973
3      0.001455 -0.003419      17        1   1.680973
44     0.001457 -0.003419      18        1   1.680973
46     0.001457 -0.003419      19        1   1.680973
13     0.001457 -0.003419      20        1   1.680973
5      0.001458 -0.003419      21        1   1.680973
8      0.001456 -0.003419      22        1   1.680971
1      0.001459 -0.003418      23        1   1.680971
47     0.001455 -0.003419      24        1   1.680971
41     0.001458 -0.003418      25        1   1.680971
22     0.001455 -0.003419      26        1   1.680971
26     0.001455 -0.003419      27        1   1.680970
12     0.001455 -0.003419      28        1   1.680970
25     0.001458 -0.003419      29        1   1.680969
6      0.001456 -0.003419      30        1   1.680969
39     0.001458 -0.003419      31        1   1.680969
28     0.001458 -0.003419      32        1   1.680969
34     0.001459 -0.003418      33        1   1.680969
24     0.001455 -0.003419      34        1   1.680969
14     0.001457 -0.003419      35        1   1.680968
4      0.001456 -0.003419      36        1   1.680967
40     0.001456 -0.003419      37        1   1.680967
15     0.001458 -0.003419      38        1   1.680967
30     0.001456 -0.003419      39        1   1.680967
11     0.001457 -0.003419      40        1   1.680967
23     0.001455 -0.003419      41        1   1.680966
45     0.001456 -0.003419      42        1   1.680966
17     0.001459 -0.003418      43        1   1.680966
48     0.001456 -0.003419      44        1   1.680964
31     0.001458 -0.003419      45        1   1.680963
36     0.001456 -0.003419      46        1   1.680962
27     0.001456 -0.003419      47        1   1.680962
20     0.001458 -0.003419      48        1   1.680961
16     0.001456 -0.003419      49        1   1.680961
21     0.001455 -0.003419      50        1   1.680957, topic_info=          Term       Freq      Total Category  logprob  loglift
6          der  81.000000  81.000000  Default  21.0000  21.0000
7     internet  46.000000  46.000000  Default  20.0000  20.0000
1         kein  42.000000  42.000000  Default  19.0000  19.0000
5        nicht  47.000000  47.000000  Default  18.0000  18.0000
14     storung  44.000000  44.000000  Default  17.0000  17.0000
..         ...        ...        ...      ...      ...      ...
736   anschluß   0.040109   2.155018  Topic50  -7.5101   0.1018
104   leistung   0.039722   2.149901  Topic50  -7.5198   0.0945
213  glasfaser   0.040317   2.396378  Topic50  -7.5050   0.0009
318  telefonie   0.040729   4.114006  Topic50  -7.4948  -0.5294
274        weg   0.039818   2.729544  Topic50  -7.5174  -0.1418

[2412 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
115       1  0.514662        aber
115       2  0.171554        aber
84        4  0.741890   anschluss
178       2  0.520419        auch
108       3  0.328414     ausfall
...     ...       ...         ...
107       2  0.155718      wieder
16        2  0.893939        wlan
282

##### May

In [22]:
custom_lda.add_corpus(token_data["may"], "may")
custom_lda.visualize("may")

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.122086 -0.295445       1        1  10.403612
32     0.141218  0.312399       2        1   6.047095
37    -0.336073  0.041759       3        1   3.952241
38     0.032688  0.056192       4        1   1.917577
42    -0.021641  0.026711       5        1   1.829662
9     -0.002380  0.008810       6        1   1.806686
27     0.001458 -0.003419       7        1   1.682810
25     0.001458 -0.003419       8        1   1.682809
20     0.001459 -0.003418       9        1   1.682807
44     0.001459 -0.003418      10        1   1.682805
2      0.001458 -0.003419      11        1   1.682805
11     0.001459 -0.003418      12        1   1.682805
21     0.001456 -0.003419      13        1   1.682803
0      0.001455 -0.003419      14        1   1.682802
46     0.001457 -0.003419      15        1   1.682802
34     0.001457 -0.003419      16        1   1.682802
41     0.001455 -0.003419      17        1   1.682802
17     0.001457 -0.003419      18        1   1.682801
39     0.001457 -0.003419      19        1   1.682800
1      0.001457 -0.003419      20        1   1.682800
29     0.001458 -0.003419      21        1   1.682800
13     0.001456 -0.003419      22        1   1.682800
15     0.001459 -0.003418      23        1   1.682800
3      0.001455 -0.003419      24        1   1.682799
47     0.001458 -0.003418      25        1   1.682799
22     0.001455 -0.003419      26        1   1.682799
36     0.001455 -0.003419      27        1   1.682799
23     0.001455 -0.003419      28        1   1.682798
48     0.001458 -0.003419      29        1   1.682797
10     0.001456 -0.003419      30        1   1.682797
12     0.001458 -0.003419      31        1   1.682797
8      0.001458 -0.003419      32        1   1.682797
31     0.001459 -0.003418      33        1   1.682797
4      0.001455 -0.003419      34        1   1.682797
24     0.001457 -0.003419      35        1   1.682796
43     0.001456 -0.003419      36        1   1.682796
5      0.001456 -0.003419      37        1   1.682796
40     0.001458 -0.003419      38        1   1.682796
28     0.001456 -0.003419      39        1   1.682795
18     0.001457 -0.003419      40        1   1.682795
49     0.001455 -0.003419      41        1   1.682795
6      0.001456 -0.003419      42        1   1.682795
16     0.001459 -0.003418      43        1   1.682794
35     0.001456 -0.003419      44        1   1.682794
26     0.001458 -0.003419      45        1   1.682794
33     0.001456 -0.003419      46        1   1.682793
30     0.001456 -0.003419      47        1   1.682793
45     0.001458 -0.003419      48        1   1.682792
14     0.001456 -0.003419      49        1   1.682791
19     0.001455 -0.003419      50        1   1.682785, topic_info=          Term       Freq      Total Category  logprob  loglift
6          der  75.000000  75.000000  Default  21.0000  21.0000
7     internet  44.000000  44.000000  Default  20.0000  20.0000
1         kein  40.000000  40.000000  Default  19.0000  19.0000
5        nicht  44.000000  44.000000  Default  18.0000  18.0000
14     storung  40.000000  40.000000  Default  17.0000  17.0000
..         ...        ...        ...      ...      ...      ...
736   anschluß   0.037517   2.011921  Topic50  -7.5101   0.1027
104   leistung   0.037154   2.006542  Topic50  -7.5198   0.0956
213  glasfaser   0.037710   2.234097  Topic50  -7.5050   0.0031
318  telefonie   0.038096   3.858518  Topic50  -7.4948  -0.5332
274        weg   0.037246   2.544749  Topic50  -7.5174  -0.1395

[2412 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
115       1  0.554272        aber
115       2  0.184757        aber
84        4  0.800237   anschluss
178       2  0.555345        auch
108       3  0.347697     ausfall
...     ...       ...         ...
107       2  0.167796      wieder
16        2  0.871520        wlan
282

### NMF

In [11]:
# Fit NMF model
nmf_model = gensim.models.Nmf(
    corpus = corpus,     # Document-Term Matrix
    id2word = id2word,   # Map word IDs to words
    num_topics = N_TOPICS,     # Number of latent topics to extract
    random_state = SEED,
    passes = 100,        # N° of passes through the corpus during training
    )

# Get the topics sorted by sparsity
pd.DataFrame(nmf_model.show_topics(), columns=["topic", "keywords"])

,topic,keywords
0,17,"0.365*""der"" + 0.015*""nicht"" + 0.013*""beheben"" ..."
1,18,"0.119*""techniker"" + 0.109*""zu"" + 0.066*""kommen..."
2,6,"0.289*""ein"" + 0.040*""techniker"" + 0.026*""haben..."
3,19,"0.314*""mein"" + 0.043*""nicht"" + 0.019*""auf"" + 0..."
4,11,"0.146*""nicht"" + 0.125*""da"" + 0.125*""warum"" + 0..."
5,14,"0.373*""nicht"" + 0.311*""funktionieren"" + 0.022*..."
6,3,"0.364*""festnetz"" + 0.056*""mein"" + 0.027*""mit"" ..."
7,16,"0.332*""gehen"" + 0.314*""nicht"" + 0.015*""telefon..."
8,1,"0.419*""wlan"" + 0.054*""verbindung"" + 0.015*""anz..."
9,7,"0.489*""telefon"" + 0.012*""beispielort"" + 0.012*..."


## Sklearn

In [12]:
vectorizer = TfidfVectorizer(stop_words=STOP_WORDS)

In [13]:
model_nmf = NMF(n_components=N_TOPICS, random_state=SEED)
model_lda = LatentDirichletAllocation(n_components=N_TOPICS, random_state=SEED)

### NMF

In [14]:
topic_pipeline = make_topic_pipeline(vectorizer, model_nmf)
topic_pipeline.fit(docs)
topicwizard.visualize(docs, model=topic_pipeline)

Preprocessing
Inferring topical content for documents.


### LDA

In [15]:
topic_pipeline = make_topic_pipeline(vectorizer, model_lda)
topic_pipeline.fit(docs)
topicwizard.visualize(docs, model=topic_pipeline)

Preprocessing
Inferring topical content for documents.
